# AO Data Slam: Explore Data and Fit Baseline Model

Peter Leimbigler

2023-02-18

# Imports and Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport
%matplotlib inline

In [2]:
plt.rcParams['figure.facecolor'] = 'w'

# Load and Explore Data

In [3]:
df = pd.read_csv('data/AO_2023_Training_file.csv')
df.shape

(105420, 20)

In [4]:
df.head()

,CAX_ID,gender,match_id,server_id,receiver_id,server_score,receiver_score,set_number,game_number,point,court_side,serve,in_play,in_net,receiver_position_x,outcome,point_winner,server_handedness,receiver_handedness,zone
0,1,men,888534,2efb1dd19b,bbffd1708b,0,0,1,1,1,Deuce,1,True,False,14.133262,winner,2efb1dd19b,Right,Right,0
1,2,men,888534,2efb1dd19b,bbffd1708b,15,0,1,1,2,Ad,1,False,False,14.119507,fault,2efb1dd19b,Right,Right,-1
2,3,men,888534,2efb1dd19b,bbffd1708b,15,0,1,1,2,Ad,2,True,False,13.676787,forced_error,2efb1dd19b,Right,Right,1
3,4,men,888534,2efb1dd19b,bbffd1708b,30,0,1,1,3,Deuce,1,True,False,14.155067,ace,2efb1dd19b,Right,Right,5
4,5,men,888534,2efb1dd19b,bbffd1708b,40,0,1,1,4,Ad,1,True,False,14.038913,unforced_error,bbffd1708b,Right,Right,1


In [5]:
df['server_score'].value_counts()

0     26463
40    25316
15    24421
30    21944
AD     4416
2       541
1       536
3       474
5       392
4       384
6       308
7       105
8        72
9        24
10       10
11        7
12        3
13        2
14        2
Name: server_score, dtype: int64

In [6]:
df['receiver_score'].value_counts()

0     38207
15    26284
40    18071
30    17081
AD     2827
1       539
2       539
3       479
4       448
5       372
6       328
7       117
8        63
9        34
10       11
11       11
12        4
13        2
14        2
16        1
Name: receiver_score, dtype: int64

In [7]:
# profile = ProfileReport(df)

In [8]:
# profile

# Prepare Data

**TODO:**
- featurize non-numeric columns
- convert target column to categorical

In [9]:
# Quick and dirty: grab numeric columns, except the ID column
X_with_nan = df.drop(columns='zone').select_dtypes('number').drop(columns='CAX_ID')

In [10]:
X_with_nan

,set_number,game_number,point,serve,receiver_position_x
0,1,1,1,1,14.133262
1,1,1,2,1,14.119507
2,1,1,2,2,13.676787
3,1,1,3,1,14.155067
4,1,1,4,1,14.038913
...,...,...,...,...,...
105415,3,10,2,2,-11.501250
105416,3,10,3,1,-12.777177
105417,3,10,4,1,-13.085640
105418,3,10,4,2,-11.506534


In [11]:
# How many rows have any missing numeric values?
rows_with_nan = X_with_nan.isna().any(axis=1).sum()
frac_rows_with_nan = rows_with_nan / X_with_nan.shape[0]
print(f'{rows_with_nan} rows with any NaN in a numeric column ({frac_rows_with_nan:.2%})')

2158 rows with any NaN in a numeric column (2.05%)


In [23]:
# FIXME: impute NaN instead of dropping it
X = X_with_nan.dropna().values

In [13]:
y = df.loc[X.index, 'zone'].values

# Baseline 1: Logistic Regression

In [14]:
from sklearn.linear_model import LogisticRegressionCV

In [24]:
model_logreg = LogisticRegressionCV(cv=5, random_state=0, verbose=1).fit(X, y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           42     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.60750D+05    |proj g|=  6.21064D+04

At iterate   50    f=  1.51353D+05    |proj g|=  9.28197D+02


/opt/miniconda3/envs/ao-data-slam/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
 This problem is unconstrained.



At iterate  100    f=  1.51271D+05    |proj g|=  1.63505D+02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   42    100    106      1     0     0   1.635D+02   1.513D+05
  F =   151270.87312570933     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           42     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.50469D+05    |proj g|=  2.49570D+03

At iterate   50    f=  1.48620D+05    |proj g|=  9.20357D+02

At iterate  100    f=  1.48541D+05    |proj g|=  9.08468D+02

       

 This problem is unconstrained.


...

 This problem is unconstrained.



At iterate   50    f=  1.45212D+05    |proj g|=  8.30719D+02

At iterate  100    f=  1.45199D+05    |proj g|=  3.01939D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   42    100    106      1     0     0   3.019D+01   1.452D+05
  F =   145199.35708767531     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           42     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.45129D+05    |proj g|=  3.02270D+01


 This problem is unconstrained.



At iterate   50    f=  1.45128D+05    |proj g|=  4.19749D+01

At iterate  100    f=  1.45128D+05    |proj g|=  1.50509D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   42    100    106      1     0     0   1.505D+01   1.451D+05
  F =   145127.53277755767     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           42     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.45118D+05    |proj g|=  1.50487D+01

           * * *

Tit   = total number of iterations
Tnf   = total nu

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   38.1s finished
/opt/miniconda3/envs/ao-data-slam/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [27]:
model_logreg.predict_proba(X[0:1, :])

array([[0.3437937 , 0.16628699, 0.12801322, 0.04290588, 0.04184625,
        0.13498534, 0.14216861]])